In [1]:
from nepal.datasets import NYTimes, Vaccinations, CountyDistance, GovernmentResponse

In [2]:
# nytimes = NYTimes().load()
# vaccinations = Vaccinations().load()
# distances = CountyDistance(radius=100).load()
response_tracker = GovernmentResponse().load()